A classe PortfólioEnv é onde o robô vai operar e tomar suas decisões. Ela recebe como parâmetros para aprender:
self, que é obrigatório em Python,
rentabilidade,
média da rentabilidade,
média de volatilidade,
drawdown,
regime (que indica como o mercado está, por exemplo: otimista, pessimista, alta volatilidade etc.),
e clusters, que representam agrupamentos por setor.

Rentabilidade (Retorno): É o "quanto variou". Se a ação valia R$ 100 e foi para R$ 110, o retorno é 0.10 (ou 10%). Se caiu para R$ 90, é -0.10.

Volatilidade: É o "quanto chacoalha".

Uma ação que faz +0.1%, -0.1%, +0.2% tem baixa volatilidade (calma).

Uma ação que faz +5%, -10%, +15% tem alta volatilidade (nervosa/arriscada).

3. Pesos: Confiança ou Aposta?
Você perguntou: "peso é confiança do modelo certo? 25% significa que ele ta confiante nessa quantidade para aposta naquela ação"

Resposta: Sim e Não.

Tecnicamente: O peso é a Alocação de Capital. É a "Aposta" física. Significa: "Do dinheiro que eu tenho no bolso, vou colocar 25% aqui".

Estrategicamente: Sim, reflete a Confiança.

Se o modelo (Rede Neural) tem certeza absoluta que a ação vai subir, ele vai tentar jogar o peso para 1.0 (100% do dinheiro).

Se ele está em dúvida ou acha que vai cair, ele diminui o peso para 0.0 ou algo muito baixo.

Então, o peso é a materialização da confiança.

Cuidado: O modelo sempre tenta maximizar o lucro. Se você não colocar limites, ele pode ter "excesso de confiança" e colocar 100% em uma ação arriscada. Por isso, em sistemas reais, costumamos limitar os pesos (ex: no máximo 30% por ação) para forçar ele a diversificar, mesmo que ele esteja "muito confiante".

In [1]:
from teste import (
    TICKERS, NUM_ATIVOS, ret, r_media, v_media, dd,
    regimes, regime_ids, cluster_ids,
    discretizar_estado_financeiro,
    calcular_recompensa_portfolio,
    aplicar_acao_portfolio
)
import numpy as np
import torch

#? É o "mundo" onde o seu robô (agente) vai viver, tomar decisões e aprender.
class PortfolioEnv:
    def __init__(self, ret, r_media, v_media, dd, regime_ids, cluster_ids):
        self.ret = ret
        self.r_media = r_media
        self.v_media = v_media
        self.dd = dd
        self.regime_ids = regime_ids
        self.cluster_ids = cluster_ids
        
        # len igual tamanho total ou seja se temos dados de 3 dias o len conta as linha e retorna e nesse explemo o jogo do robo duraria 3 estado
        self.num_steps = len(ret) # # Duração total do jogo (ex: 1000 dias)
        
        # .shape (forma) devolve as dimensões da tabela no formato (linhas, colunas).Quando usamos o indice 1 pegamos o números de colunas
        self.num_assets = ret.shape[1] # # Quantas ações temos (ex: 10 ações)
        self.t = 1
        
        self.valor_carteira = 1.0
        self.pico_historico = 1.0
        # np.ones(N) cria uma lista de tamanho N preenchida apenas com o número 1.
        # np.ones(4): O computador cria isso: [1, 1, 1, 1] / self.num_assets: Ele divide todos os números por 4. Ou seja inicia o jogo com dinheiro dividido igualmente entre as ações
        self.pesos = np.ones(self.num_assets) / self.num_assets

    #? Reiniciando a Partida
    def reset(self):
        self.t = 1 # inciia o jogo no dia 1
        self.pesos = np.ones(self.num_assets) / self.num_assets # Divide dinheiro igualmente entre as ações

        self.valor_carteira = 1.0  # Começa com 100% do capital (normalizado em 1.0)
        self.pico_historico = 1.0  # O topo histórico inicial é o próprio valor inicial
        
        # seria a visão do robô
        state = self._get_state()
        return state

    # O "Dia a Dia" (A parte mais importante)
    def step(self, novos_pesos):
        # 1. Pega os retornos do mercado hoje
        retornos_dia = self.ret.iloc[self.t].values

        # --- CÁLCULO DA CARTEIRA DO ROBÔ (A Lógica Nova) ---
        
        # Calcula quanto a carteira rendeu com os novos pesos
        # Nota: Estamos assumindo que ele rebalanceou e pegou o retorno do dia (simplificação comum em RL)
        r_p = np.dot(novos_pesos, retornos_dia)

        # Atualiza o valor total da carteira (Juros compostos)
        self.valor_carteira *= (1 + r_p)

        # Atualiza o topo histórico (High Water Mark)
        if self.valor_carteira > self.pico_historico:
            self.pico_historico = self.valor_carteira

        # Calcula o Drawdown REAL dele (Quanto caiu desde o topo?)
        # Se valor = 0.95 e pico = 1.00 -> 0.95/1.0 - 1 = -0.05 (-5%)
        meu_drawdown = (self.valor_carteira / self.pico_historico) - 1
        
        # ----------------------------------------------------

        pesos_antigos = self.pesos.copy()

        # Calcula recompensa usando o SEU drawdown e o lambda ajustado
        reward = calcular_recompensa_portfolio(
            pesos_antigos=pesos_antigos,
            pesos_novos=novos_pesos,
            retornos_dia=retornos_dia,
            drawdown_dia=meu_drawdown,  # <--- AGORA USA O DD DO ROBÔ
            lambda_dd=0.02,              # <--- PUNICAO JUSTA (0.2)
            lambda_tc=0.001
        )

        # Atualiza estado
        self.pesos = novos_pesos
        self.t += 1
        done = (self.t >= self.num_steps - 1)

        state = self._get_state()
        return state, reward, done

    #? A "Visão" do Robô
    def _get_state(self):

        # o que o robô enxerga é o passado (t-1)
        t_obs = self.t - 1
        # np.concatenate unir vários array e claro a gente transforam todos eles no tipo float32
        state = np.concatenate([
            self.pesos, # Como está minha carteira agora?
            [self.r_media.iloc[t_obs]], # Qual a média de retorno recente?
            [self.v_media.iloc[t_obs]], # Qual a volatilidade recente?
            [self.dd.iloc[t_obs]], 
            np.array(self.cluster_ids) # A qual setor cada ação pertence?
        ]).astype(np.float32)
  
        state = np.nan_to_num(state, nan=0.0, posinf=0.0, neginf=0.0)
        return state

Arquivo encontrado! Carregando dados locais...
            BOVA11.SA  SMAL11.SA  IVVB11.SA  PETR4.SA   VALE3.SA  ITUB4.SA  \
Date                                                                         
2015-01-01        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-02  47.259998  52.020000  55.799999  2.572509  10.505502  9.372999   
2015-01-03        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-04        NaN        NaN        NaN       NaN        NaN       NaN   
2015-01-05  46.320000  50.549999  55.750000  2.352637  10.347521  9.420101   

                   SPY        QQQ         IWM        EEM         GLD  \
Date                                                                   
2015-01-01         NaN        NaN         NaN        NaN         NaN   
2015-01-02  171.093704  94.906532  103.315140  30.789457  114.080002   
2015-01-03         NaN        NaN         NaN        NaN         NaN   
2015-01-04         NaN        NaN         NaN 

d:\python\portfolioDeAtivos\teste.py:154: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  retornos = precos.pct_change().fillna(0)


-0.008030805295683797
Soma: 1.0000000000000002
Min: 0.05767012687427913
Max: 0.07727797001153404
[0.07727797 0.05767013 0.05767013 0.05767013 0.05767013]
Ação 0 | soma=1.000000 | min=0.0577 | max=0.0773
Ação 1 | soma=1.000000 | min=0.0565 | max=0.0761
Ação 2 | soma=1.000000 | min=0.0554 | max=0.0750
Ação 3 | soma=1.000000 | min=0.0543 | max=0.0740
Ação 4 | soma=1.000000 | min=0.0533 | max=0.0729
Ação 5 | soma=1.000000 | min=0.0522 | max=0.0718
Ação 6 | soma=1.000000 | min=0.0512 | max=0.0708
Ação 7 | soma=1.000000 | min=0.0502 | max=0.0698
Ação 8 | soma=1.000000 | min=0.0492 | max=0.0688
Ação 9 | soma=1.000000 | min=0.0483 | max=0.0679


PolicyMLP = O Ator (O Jogador)

O que faz:
    Ele olha para o estado do jogo e decide qual ação tomar.

Saída:
    Probabilidades (Ex: 80% pular, 20% correr).

Objetivo:
    Aprender a escolher as ações que dão mais recompensa.

No código:
    É por isso que termina com softmax. Ele precisa escolher uma ação.


ValueMLP = O Crítico (O Treinador)

O que faz:
    Ele não joga. Ele olha para o estado e diz "quão boa é essa situação".

Saída:
    Um único número real (chamado de Valor ou V(s)).

Exemplo:
    Se o boneco está prestes a cair num buraco:
        Valor baixo (ex: -100).

    Se está perto de ganhar:
        Valor alto (ex: 500).

Por que treinamos ele?
    Para "guiar" o Ator.

    Muitas vezes, a recompensa do jogo demora para chegar
    (só ganha ponto no final da fase). O Ator ficaria perdido
    sem saber se está indo bem.

    O ValueMLP aprende a prever essa recompensa futura.
    Ele serve como uma bússola imediata para o Ator.


Resumido um modelo para jogar e outro modelo para avaliar o desempenho do jogador

In [2]:


#saida = x * W + b
# ativação relu = max(0, saida) Ela zera valores negativos e deixa positivos como estão.
import torch.nn as nn
# Isso cria uma rede neural personalizada herdando de nn.Module
class PolicyMLP(nn.Module):
# state_dim é o tamanho do vetor de entrada (dimensão do estado). num_actions é quantas ações o agente pode tomar
    def __init__(self, state_dim, num_actions):
# O super() só registra o módulo na infraestrutura do PyTorch
        super().__init__()
        self.net = nn.Sequential(
# na camada linear: estado → 128 neurônios.Primeiro parametro é entrada e a segunda é quantidade de neuronios
            nn.Linear(state_dim, 128), 
# ReLU como ativação           
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_actions)
        )
#A entrada x passa pela rede e vira logits.Logits são valores crus, sem normalização
    def forward(self, x):
        logits = self.net(x)
# softmax converte isso numa distribuição de probabilidade sobre as ações (cada linha soma 1).dim=1 significa: aplicar softmax ao longo das colunas, isto é, entre ações.
        return logits
    
# Qual bom ele está nessse estado ? 
class ValueMLP(nn.Module):
    def __init__(self, state_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(state_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

    def forward(self, x):
        return self.net(x)

